# Semi-structured eval: baseline retriever

We will test retrieval of table information from the `Semi-structured Reports` dataset using various methods.

## Pre-requisites

In [8]:
%pip install --quiet -U langchain langsmith langchain_benchmarks
%pip install --quiet chromadb openai pypdf pandas

Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
types-requests 2.31.0.10 requires urllib3>=2, but you have urllib3 1.26.18 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


## Dataset

In [9]:
import os

from langchain_benchmarks import registry
from langchain_benchmarks.rag.tasks.semi_structured_reports import get_file_names

# Task
task = registry["Semi-structured Reports"]

# Files used
paths = list(get_file_names())
files = [str(p) for p in paths]

### TODO: Replace when dataset is updated
#dir = "/Users/rlm/Desktop/Eval_Sets/semi_structured_reports/"
dir = "/mnt/c/Users/taqi_/OneDrive/Desktop/semi_structured_reports/"
files = [f for f in os.listdir(dir) if f.endswith(".pdf")]

## Load

In [10]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


def load_and_split(file,token_count):
    """
    Load and split PDF files
    """

    loader = PyPDFLoader(file)
    pdf_pages = loader.load()

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=token_count, chunk_overlap=50
    )

    # Get chunks
    docs = text_splitter.split_documents(pdf_pages)
    texts = [d.page_content for d in docs]
    print(f"There are {len(texts)} text elements")
    return texts


texts = []
token_count = 1500
for fi in files:
    texts.extend(load_and_split(dir + fi,token_count))

There are 18 text elements
There are 4 text elements
There are 11 text elements
There are 11 text elements
There are 14 text elements
There are 3 text elements


## Index

In [11]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

vectorstore_baseline = Chroma.from_texts(
    texts=texts, collection_name="baseline-1500-token", embedding=OpenAIEmbeddings()
)

retriever_baseline = vectorstore_baseline.as_retriever()

## RAG

In [12]:
from typing import List

from langchain.schema.document import Document
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

from langchain.cache import SQLiteCache
from langchain.globals import set_llm_cache

set_llm_cache(SQLiteCache(database_path="/tmp/.langchain.db"))


def format_context(docs: List[Document]) -> str:
    return "\n\n".join([i.page_content for i in docs])


def rag_chain(retriever):
    """
    RAG chain
    """

    # Prompt template
    template = """Answer the question based only on the following context, which can include text and tables:
    {context}
    Question: {question}
    """
    prompt = ChatPromptTemplate.from_template(template)

    # LLM
    model = ChatOpenAI(temperature=0, model="gpt-4")

    # RAG pipeline
    chain = (
        {
            "context": retriever | RunnableLambda(format_context),
            "question": RunnablePassthrough(),
        }
        | prompt
        | model
        | StrOutputParser()
    )
    return chain


# Create RAG chain
chain = rag_chain(retriever_baseline)

## Eval

In [13]:
### TODO: Replace with public dataset

import uuid

import pandas as pd
from langsmith import Client

# Read
df = pd.read_csv(dir + "semi_structured_reports.csv")

# Dataset
client = Client()
dataset_name = f"Semi-Structured-Eval-v5"
existing_datasets = list(client.list_datasets(dataset_name=dataset_name))
if existing_datasets:
    # read existing dataset
    dataset = client.read_dataset(dataset_name=dataset_name)
else:
    dataset = client.create_dataset(dataset_name=dataset_name)
    # Populate dataset
    for _, row in df.iterrows():
        # Get Q, A
        q = row["Question"]
        a = row["Answer"]

        # Use the values in your function
        client.create_example(
            inputs={"question": q}, outputs={"answer": a}, dataset_id=dataset.id
        )

In [14]:
import uuid
from langsmith.client import Client
from langchain.smith import RunEvalConfig

eval_config = RunEvalConfig(
    evaluators=["cot_qa"],
)

def run_eval(chain, eval_run_name):
    """
    Run eval
    """
    client = Client()
    test_run = client.run_on_dataset(
        ### TODO: Replace with public dataset
        dataset_name="Semi-Structured-Eval-v5",
        llm_or_chain_factory=lambda: (lambda x: x["question"]) | chain,
        evaluation=eval_config,
        verbose=True,
        project_name=eval_run_name,
    )


# Experiments
chain_map = {
    "baseline-1500-tok": chain,
}

run_id = str(uuid.uuid4())
for project_name, chain in chain_map.items():
    run_eval(chain, project_name + "_" + run_id)

View the evaluation results for project 'baseline-1500-tok_273fbc61-6aca-4c08-bf1d-3fb858b8bdfe' at:
https://smith.langchain.com/o/530c4d06-5640-4c0f-94fe-0be7b769531f/datasets/a89c07d6-a467-414e-9349-8e49870c99bb/compare?selectedSessions=2b3171ec-debd-4ade-b0f3-808dcb1bb02e

View all tests for Dataset Semi-Structured-Eval-v5 at:
https://smith.langchain.com/o/530c4d06-5640-4c0f-94fe-0be7b769531f/datasets/a89c07d6-a467-414e-9349-8e49870c99bb
[------------------------------------------------->] 25/25

,output,feedback.COT Contextual Accuracy,error,execution_time
count,25,25.000000,0,25.000000
unique,25,NaN,0,NaN
top,Datadog's Non-GAAP gross margin for the Nine M...,NaN,NaN,NaN
freq,1,NaN,NaN,NaN
mean,NaN,0.680000,NaN,6.216445
std,NaN,0.476095,NaN,2.818130
min,NaN,0.000000,NaN,3.314484
25%,NaN,0.000000,NaN,4.964752
50%,NaN,1.000000,NaN,5.187304
75%,NaN,1.000000,NaN,6.394062
